In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import progressbar
import time
from tqdm import tqdm
!pip install pytorch-model-summary
from pytorch_model_summary import summary
import os
import copy

if torch.cuda.is_available():
  device = 'cuda:0'
else:
  device = 'cpu'

from torchvision import models

resnet_50 = models.resnet50(pretrained=True)
for param in resnet_50.parameters():
    param.requires_grad = True
#print(summary(resnet_50, torch.zeros((5, 3, 64, 64)), show_input=False))

In [ ]:
print(list(resnet_50.children())[:-5])

In [ ]:
import os
os.chdir('/content/')
#!rm -r ImageNet 
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
# ! wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2.tgz
!ls
!tar -xzf imagenette2.tgz
# !tar -xzf imagewoof2.tgz
# !mv imagewoof2 imagenette2
!mkdir ImageNet
!mv imagenette2/ ImageNet/
!mv ImageNet/imagenette2/ ImageNet/train
# !rm -r tiny-imagenet-200/test
!mkdir ImageNet/test
#!mkdir

--2020-12-16 06:37:49--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.109.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.109.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1556914727 (1.4G) [application/x-tar]
Saving to: ‘imagenette2.tgz’

imagenette2.tgz     100%[===================>]   1.45G  46.0MB/s    in 33s     

2020-12-16 06:38:22 (44.8 MB/s) - ‘imagenette2.tgz’ saved [1556914727/1556914727]

imagenette2.tgz  sample_data


In [ ]:
# import os
# os.chdir('/content/')
# !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
# !ls
# !unzip tiny-imagenet-200.zip
# !rm -r tiny-imagenet-200/test
# !mkdir tiny-imagenet-200/test

In [ ]:
'''
import os,shutil
import numpy as np
path = '/content/Caltech101/'
os.chdir(path)
# os.listdir(path+'train')
for name in os.listdir(path+'train'):
	if not(os.path.exists(path+'test/'+name)):
		os.mkdir(path+'test/'+name)

for name in os.listdir(path+'train'):
	sz = int(len(os.listdir(path+'train/'+name+'/'))/5)
	# l = np.random.choice(lst,sz).copy()
	# print(len(l),name)
	for i in range(sz):
		# file = i
		lst = os.listdir(path+'train/'+name+'/')
		file = np.random.choice(lst)
		shutil.move(path+'train/'+name+'/'+file,path+'test/'+name+'/'+file)
'''

In [ ]:
# import os,shutil
# import numpy as np
# path = '/content/tiny-imagenet-200/'
# os.chdir(path)
# # os.listdir(path+'train')
# for name in os.listdir(path+'train'):
# 	if not(os.path.exists(path+'test/'+name)):
# 		os.mkdir(path+'test/'+name)

# for name in os.listdir(path+'train'):
# 	sz = int(len(os.listdir(path+'train/'+name+'/images/'))/5)
# 	# l = np.random.choice(lst,sz).copy()
# 	# print(len(l),name)
# 	for i in range(sz):
# 		# file = i
# 		lst = os.listdir(path+'train/'+name+'/images/')
# 		file = np.random.choice(lst)
# 		shutil.move(path+'train/'+name+'/images/'+file,path+'test/'+name+'/'+file)

In [ ]:
train_path = '/content/ImageNet/train/train/'
#def train_dataloader():
train_transform = transforms.Compose([transforms.Resize((224,224)),transforms.RandomHorizontalFlip(),transforms.RandomRotation(15),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
# train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
valid_transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
train_data = torchvision.datasets.ImageFolder(root=train_path,transform=train_transform)
#dataloader = torch.utils.data.DataLoader(data,batch_size=4,shuffle=True)
#return dataloader

test_path = '/content/ImageNet/train/val/'
test_data = torchvision.datasets.ImageFolder(root = test_path,transform=valid_transform)

datasets = {'train':train_data,'valid':test_data}

In [ ]:
# train_path = '/content/tiny-imagenet-200/train/'
# #def train_dataloader():
# train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomRotation(15),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
# # train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
# valid_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
# train_data = torchvision.datasets.ImageFolder(root=train_path,transform=train_transform)
# #dataloader = torch.utils.data.DataLoader(data,batch_size=4,shuffle=True)
# #return dataloader

# test_path = '/content/tiny-imagenet-200/test/'
# test_data = torchvision.datasets.ImageFolder(root = test_path,transform=valid_transform)

# datasets = {'train':train_data,'valid':test_data}

In [ ]:
def get_dist_mat(coords_org):
    coords=coords_org.clone().detach()						
    n=coords.shape[1]											                  #coords_org = b,n,2
    tid=torch.eye(n)
    tid=tid.to(coords_org.device)
    tid.requires_grad=False

    dist_mat=coords.clone()										              #dist_mat = b,n,2
    dist_mat=dist_mat.unsqueeze(2)								          #dist_mat = b,n,1,2
    coords=coords.unsqueeze(2)									            #coords = b,n,1,2
    #print(coords.shape,n)
    for i in range(n-1):
        dist_mat=torch.cat([dist_mat,coords],dim=2)				  #dist_mat = b,n,n,2
        
    #print(dist_mat.shape)
    #print(tid.shape)
    dist_mat = dist_mat.sub(coords.transpose(1,2))				  #dist_mat = b,n,n,2
    dist_mat = torch.square(dist_mat)							
    dist_mat = torch.sum(dist_mat,dim=3)						        #dist_mat = b,n,n
    dist_mat = torch.cuda.FloatTensor(dist_mat.float())
    dist_mat = torch.sqrt(dist_mat) + tid
    
    
    #dist_mat[dist_mat<=1]=1
    #dist_mat[torch.logical_and((dist_mat>1),(dist_mat<=10))]=0.5
    #dist_mat[torch.logical_and((dist_mat>10),(dist_mat<=40))]=0.25
    #dist_mat[dist_mat>40]=0
    
    
    dist_mat = 1/dist_mat
    #dist_mat = dist_mat - tid
    dist_mat = dist_mat/torch.max(dist_mat)
    dist_mat = dist_mat-tid*dist_mat
    dist_mat = dist_mat+tid*(torch.max(dist_mat))
    
    
    #dist_mat=dist_mat.detach()
    dist_mat.requires_grad=False
    
    return dist_mat


In [ ]:
class Graph_Conv(nn.Module):
  def __init__(self,in_features,out_features):
    super(Graph_Conv, self).__init__()
    self.weight = Parameter(torch.FloatTensor(in_features, out_features))
    torch.nn.init.xavier_normal_(self.weight, gain=1.0)
#     torch.nn.init.normal_(self.weight, mean=0, std=1)

  def forward(self,x,A):      #x = b,d,nodes ; #A = b,nodes,nodes ; weight = d,d'
    #apply 1 graph convs(normal nn)
    # wei = torch.mm(x,self.weight)
    # lay = torch.mm(A,wei)     
    # lay = nn.functional.relu(A@(x.permute(0,2,1))@self.weight)
    lay = A@(x.permute(0,2,1))@self.weight
    lay = lay.permute(0,2,1)
    # if (self.weight.shape[0] == 256) & (self.weight.shape[1] == 256):
    #     print(self.weight)
    return lay              

In [ ]:
class delft_block(nn.Module):
  def __init__(self,input_dimensions,k):
    super(delft_block,self).__init__()
    #self.conv1 = nn.Conv2d(input_dim, 512, kernel_size=kernel_size, padding=padding, stride=stride)
    #self.activate = nn.ReLU()
    self.k = k
    self.inp = input_dimensions
    #self.conv1 = nn.Conv2d(input_dimensions,512,kernel_size=3,padding=1)
    self.conv1 = nn.Conv2d(input_dimensions, 256, kernel_size=1, padding=0, stride=1)
    self.bn1 = nn.BatchNorm2d(256)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 = nn.Conv2d(256,1, kernel_size=1, padding=0, stride=1)
    self.downsample = nn.Conv2d(input_dimensions, 1, kernel_size=1, padding=0, stride=1)
    self.bn = nn.BatchNorm2d(1)
  
  def forward(self,x):                                          #x = b,512,8,8
    b,c,h,w = x.size()
    residual = x
    out = self.conv1(x)                                         #out = b,1,8,8
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    residual = self.downsample(residual)
    resdiual = self.bn(residual)
    out += residual

    prob = nn.Softplus()(out)                                       
    x = x*prob                                                  #attention score multiplied by corresponding feature maps
    att = F.softmax(prob.view(b,-1),dim=1)                      #b,8*8(pixel wise attention scores flattened)
    #print(att.size())
    val,indices = torch.topk(att,self.k)                        # finding pixel indices with top k attention scores
    return x,indices
#     if phase == 'train':
'''
    if att.requires_grad:
        att.retain_grad()
    if indices.requires_grad:
        indices.retain_grad()
    ind_exp = indices.unsqueeze(-1).expand(b,self.k,self.inp)             
    
    l_perm = x.permute(0,2,3,1)                                 #l_perm = b,8,8,512
    l_perm_r = l_perm.reshape(b,h*w,self.inp)                     #l_perm = b,h*w,512
    feat = torch.gather(l_perm_r,1,ind_exp)                       #feat = b,k,512 using indices found earlier to extract feature maps[Should work now]
    if feat.requires_grad:
        feat.retain_grad()
    feat_n = feat.permute(0,2,1)

    return feat_n                                                 #feat = b,512,k
'''

'\n    if att.requires_grad:\n        att.retain_grad()\n    if indices.requires_grad:\n        indices.retain_grad()\n    ind_exp = indices.unsqueeze(-1).expand(b,self.k,self.inp)             \n    \n    l_perm = x.permute(0,2,3,1)                                 #l_perm = b,8,8,512\n    l_perm_r = l_perm.reshape(b,h*w,self.inp)                     #l_perm = b,h*w,512\n    feat = torch.gather(l_perm_r,1,ind_exp)                       #feat = b,k,512 using indices found earlier to extract feature maps[Should work now]\n    if feat.requires_grad:\n        feat.retain_grad()\n    feat_n = feat.permute(0,2,1)\n\n    return feat_n                                                 #feat = b,512,k\n'

In [ ]:
class mlp(nn.Module):
    def __init__(self,num_class):
        super(mlp,self).__init__()
        self.linear1 = nn.Linear(64,32)
        self.linear2 = nn.Linear(32,num_class)
        #self.linear3 = nn.Linear(32,num_class)
        #self.dropout = nn.Dropout(p=0.4)

    def forward(self,x):
        #x = self.dropout(F.relu(self.linear1(x)))
        #x = self.dropout(F.relu(self.linear2(x)))
        x = self.linear2(self.linear1(x))
        return x

In [ ]:
class Attn_Graph(nn.Module):
    def __init__(self,num_class=10,k=120):
        super(Attn_Graph, self).__init__()
        
        self.k = k
        self.feature1 = nn.Sequential(*list(resnet_50.children())[:-5])
        # self.feature2 = nn.Sequential(*list(resnet_50.children())[:-5])
        # self.delft1 = delft_block(512,self.k)
        self.delft2 = delft_block(256,self.k)
        # self.projector = nn.Conv1d(in_channels = 256,out_channels=512,kernel_size = 1)
        #self.delft2 = delft_block(256,40)
        self.gcn1 = Graph_Conv(256,256)
        self.gcn2 = Graph_Conv(256,128)
        self.gcn3 = Graph_Conv(128,128)
        self.gcn4 = Graph_Conv(128,64)
        self.mlp = mlp(num_class)
        self.bn1 = nn.BatchNorm1d(num_features = 256)
        self.bn2 = nn.BatchNorm1d(num_features = 256)
        self.bn3 = nn.BatchNorm1d(num_features = 128)

        
    def forward(self,x):
        #print(x.shape)
        
        y1 = self.feature1(x)                               #b,512,56,56
        y5,ind = self.delft2(y1)                            #b,512,k

        #y2 = self.feature2(x)                     #b,256,16,16
        #y2 = self.projector(y2)                   #b,512,16,16
        #y2 = self.delft2(y2)                            #b,256,k
        #y2 = self.projector(y2)                   #b,512,k

        ind_x = ind//56
        ind_y = ind%56
        ind_new = torch.stack((ind_x,ind_y),dim=2)

        #y5 = torch.cat((y1,y2),dim=2)             #b,512,nodes(2k)
        y5 = nn.ReLU()(y5)
        #trans = y5.permute(0,2,1)                 #b,nodes,512
        A = get_dist_mat(ind_new)

        #Normalizing adjacency matrix
        #A = A + I
        # print(A)
        #dhat = torch.diag_embed(torch.pow(torch.sum(A,2),-0.5))
        # print(dhat)
        #A = dhat@A@dhat                           #b,nodes,nodes

        #Building adjacency matrix
        
        #A = torch.matmul(trans,y5)                #b,nodes,nodes
        # b,nodes,nodes = A.size()
        # I = torch.eye(nodes).to(device)
        # I = I.reshape((1, nodes, nodes))
        # I = I.repeat(b, 1, 1)

        b,c,h,w = y5.size()
        # print(indices.shape)
        _,k = ind.shape
        # ind_exp = indices.unsqueeze(-1).expand(b,self.k,self.inp)
        ind_exp = ind.unsqueeze(-1).expand(b,k,c)             
        l_perm = y5.permute(0,2,3,1)                                        #l_perm = b,56,56,512
        l_perm_r = l_perm.reshape(b,h*w,-1)                                 #l_perm = b,h*w,512
        feat = torch.gather(l_perm_r,1,ind_exp)                             #feat = b,k,512 using indices found earlier to extract feature maps[Should work now]
        
        # feat_n = feat.permute(0,2,1) #check?
        feat = feat.permute(0,2,1)

        #Normalizing adjacency matrix
        #A = A + I
        # print(A)
        dhat = torch.diag_embed(torch.pow(torch.sum(A,2),-0.5))
        # print(dhat)
        A = dhat@A@dhat                           #b,nodes,nodes
        #print(A.shape)
        #print(A)
        

        #Graph Convolutions
#         print(y5[0,:30])
        y5 = self.gcn1(feat,A)
#         print(y5[0,:,30])
        y5 = self.bn2(y5)
        y5 = nn.ReLU()(y5)
        y5 = self.gcn2(y5,A)
        y5 = nn.ReLU()(y5)
#         print(y5[0,:,30])
        y5 = self.gcn3(y5,A)                       #b,256,nodes
        y5 = nn.ReLU()(y5)
        y5 = self.gcn4(y5,A)
#         print(y5[0,:,30])
        y6 = torch.mean(y5,2)                      #b,256
        y6 = self.mlp(y6)                          #b,classes(k = #classes)

        return y6,ind

In [ ]:
#print(summary(Attn_Graph(), torch.zeros((5, 3, 224, 224)), show_input=False))

def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            #print(n)
            
            #print(p.grad)
            
            if p.grad==None:
                print(n,p.grad)
              
            if p.grad!=None:
                print(n,"finite gradient")
                ave_grads.append(p.grad.abs().mean())
                max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    #plt.legend([Line2D([0], [0], color="c", lw=4),
    #           Line2D([0], [0], color="b", lw=4),
    #          Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])


In [ ]:
!mkdir images

mkdir: cannot create directory ‘images’: File exists


In [ ]:
class Learner():

    def __init__(self,datasets,model,criterion,optimizer,scheduler=None,bs=2,num_workers=2,device='cuda:0'):
        '''
        :param datasets: a dictionary containing the dataset classes with keys 'train' and 'valid'
        :param criterion: loss function
        :param device: torch.device()
        :param optimizer: no default value
        :param scheduler: default none
        :param bs: batch size for training
        :param num_workers: number of dataloader workers
        :return:
        '''
        self.datasets = datasets
        self.device = torch.device(device)
        self.model = model.to(self.device)
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.dataloaders = {}
        self.dataset_sizes = {}
        for key in self.datasets:
            shuffle = True if key=='train' else False
            self.dataloaders[key] = DataLoader(self.datasets[key],batch_size=bs,num_workers=num_workers,shuffle=shuffle)
            self.dataset_sizes[key] = len(self.datasets[key])

    def fit(self,tb_logs=None,epochs=10):
        '''
        :param tb_logs: a dictionary containing tensorboard log flag, path and comment
        :param epochs: number of epochs
        :return:
        '''
        # manually set image size for standard dataset class
        # imgsize = self.datasets['train'].img_size
        imgsize = self.datasets['train'][0][0].size()[2]

        if tb_logs is not None:
            logpath = tb_logs['path']
            logcomment = tb_logs['comment']
            tb = SummaryWriter(log_dir=logpath+f'/{logcomment}', comment=logcomment)

        best_model_wts = copy.deepcopy(self.model.state_dict())
        best_valid_loss=100.
        self.record_dict = {'train': {'loss': [], 'acc': []}, 'valid': {'loss': [], 'acc': []}}

        try:
          tqdm._instances.clear()
        except:
          pass

        for epoch in range(epochs):
            cnt = 0
            print(f'EPOCH : {epoch + 1}/{epochs}')
            for phase in self.dataloaders.keys():
                since = time.time()
                if phase == 'train':
                    self.model.train()
                else:
                    self.model.eval()
                running_loss = 0.
                running_acc = 0.
                #------------------Progress Bar------------------#
                # bari = 0
                # bar = progressbar.ProgressBar(maxval=self.dataset_sizes[phase], \
                # widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
                # bar.start()
                # print(bari)
                #------------------------------------------------#
                for inputs, targets in tqdm(self.dataloaders[phase]):
                    inputs = inputs.float().view(-1, 3, imgsize, imgsize).to(self.device)
                    #print(inputs)
                    # print()
                    # print()
                    #print(inputs.shape)
                    # define targets and push to device
                    targets = targets.to(self.device)
                    #print(targets.shape)
                    #targets = torch.unsqueeze(targets,-1)
                    #print(targets.shape)
                    with torch.set_grad_enabled(phase=='train'):
                        self.optimizer.zero_grad()
                        #outputs = self.model(inputs)
                        outputs,indices = self.model(inputs)

                        # visualisation block
                        ind_x = indices//56                                                  #b,k
                        ind_y = indices%56                                                   #b,k
                        # #-----------------Saliency Map--------------------#
                        if epoch%30 == 0:  
                            img = inputs[0].detach().to('cpu').numpy()
                            # img = inputs[0].detach().to('cpu').numpy()
                            img = np.transpose(img,(1,2,0))
                            indx = ind_x.detach()[0,:].to('cpu').numpy()
                            indy = ind_y.detach()[0,:].to('cpu').numpy()
                            for i in range(len(indx)):
                                x,y = indx[i],indy[i]
                                img[4*x+1,4*y+1,:] = [1,0,0]
                                img[4*x+1,4*y+2,:] = [1,0,0]
                                img[4*x+2,4*y+1,:] = [1,0,0]
                                img[4*x+2,4*y+2,:] = [1,0,0]
                            # plt.imshow(img)
                            cnt += 1

                            plt.imsave('images/{}_{}.jpg'.format(epoch,cnt), np.clip(img,0,1))
                            plt.close()
                        #-------------------------------------------------#

                        #preds = outputs
                        preds = torch.argmax(outputs,dim=1)
                        #print(preds.shape)
                        # current loss written for categorical cross entropy
                        loss = self.criterion(outputs, targets)
                        # print("LOss function",loss)
                        # print(outputs)
                        # print(targets)
                        acc = torch.mean((targets==preds).to(float)).item()
                        if phase == 'train':
                            loss.backward()
                            #plot_grad_flow(self.model.named_parameters())
                            self.optimizer.step()
                            # self.scheduler.step()
                        running_loss += loss.item() * inputs.size()[0]
                        running_acc += acc*inputs.size()[0]
        
                        # Check for scheduler first
                        # if phase == 'train' and not(scheduler is None):
                        #     scheduler.step(epoch_loss)
                    # bari+=1
                    # bar.update(bari)
                
                # bar.finish()
                epoch_loss = running_loss/self.dataset_sizes[phase]
                epoch_acc = running_acc/self.dataset_sizes[phase]

                if tb_logs is not None:
                    if phase == 'train':
                        tb.add_scalar('Train Loss', epoch_loss, epoch)
                        tb.add_scalar('Train Acc',epoch_acc,epoch)
                    else:
                        tb.add_scalar('Valid Loss', epoch_loss, epoch)
                        tb.add_scalar('Valid Acc', epoch_acc, epoch)

                # validation loss based schedulers (Reduce on plateau)
                # if phase == 'valid' and not (self.scheduler is None):  
                #     self.scheduler.step(epoch_loss)

                self.record_dict[phase]['loss'].append(epoch_loss)
                self.record_dict[phase]['acc'].append(epoch_acc)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss,epoch_acc))
                time_elapsed = time.time() - since
                print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
                # deep copy the best model
                if phase == 'valid' and epoch_loss < best_valid_loss:
                    best_valid_loss = epoch_loss
                    best_model_wts = copy.deepcopy(self.model.state_dict())
                
                path_output = '/content/ImageNet/model.pth'
                torch.save(best_model_wts, path_output)

                if tb_logs is not None:
                    pass
                    # to monitor learning rate
                    # current_lr = self.optimizer.param_groups[0]['lr']
                    # tb.add_scalar('Learning Rate', current_lr, epoch)

                    # define figures well if we want to display on tensorboard
                    # if epoch%15==0:
                    #     train_fig = 
                    #     valid_fig = 
                    #     tb.add_figure('train figs',train_fig)
                    #     tb.add_figure('valid figs',valid_fig)
        print('Best valid loss: {:4f}'.format(best_valid_loss))
        
        plt.plot(self.record_dict['train']['loss'])
        plt.savefig('/content/ImageNet/trainingloss')
        
        plt.close()
        
        plt.plot(self.record_dict['train']['acc'])
        plt.savefig('/content/ImageNet/trainingacc')
        plt.close()
        
        
        plt.plot(self.record_dict['valid']['loss'])
        plt.savefig('/content/ImageNet/Validloss')
        
        plt.close()
        
        plt.plot(self.record_dict['valid']['acc'])
        plt.savefig('/content/ImageNet/Validacc')
        plt.close()
        
        

        # load best model weights
        
        self.model.load_state_dict(best_model_wts)
        if tb_logs is not None:
            tb.close()

In [ ]:
!mkdir images

In [ ]:
# training on imagewoof dataset
print(torch.cuda.is_available())
# model = Attn_Graph()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = .0003)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
Learner_obj = Learner(datasets,model,criterion,optimizer,bs=50,num_workers=4,device=device)
epochs = 80
Learner_obj.fit(epochs=epochs)

  0%|          | 0/190 [00:00<?, ?it/s]

True
EPOCH : 1/80


  0%|          | 0/79 [00:00<?, ?it/s]

train Loss: 0.4022 Acc: 0.8667
Training complete in 1m 11s


  0%|          | 0/190 [00:00<?, ?it/s]

valid Loss: 0.7486 Acc: 0.7720
Training complete in 0m 26s
EPOCH : 2/80


 43%|████▎     | 82/190 [00:32<00:36,  2.98it/s]

KeyboardInterrupt: ignored

In [ ]:
# training on imagewoof dataset
print(torch.cuda.is_available())
model = Attn_Graph()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = .0003)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
Learner_obj = Learner(datasets,model,criterion,optimizer,bs=50,num_workers=4,device=device)
epochs = 80
Learner_obj.fit(epochs=epochs)

True


  0%|          | 0/190 [00:00<?, ?it/s]

EPOCH : 1/80


  0%|          | 0/79 [00:00<?, ?it/s]

train Loss: 1.7214 Acc: 0.3975
Training complete in 1m 10s


...

train Loss: 0.4103 Acc: 0.8634
Training complete in 1m 9s


  0%|          | 0/190 [00:00<?, ?it/s]

valid Loss: 0.5916 Acc: 0.8181
Training complete in 0m 25s
EPOCH : 15/80


 21%|██        | 40/190 [00:15<00:50,  2.98it/s]

KeyboardInterrupt: ignored

In [ ]:
# print(torch.cuda.is_available())
'''Compare with resnet_50 full '''
# model = resnet_50
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = True  
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 10)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = .0003)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
Learner_obj = Learner(datasets,model,criterion,optimizer,bs=50,num_workers=4,device=device)
epochs = 80
Learner_obj.fit(epochs=epochs)

  0%|          | 0/190 [00:00<?, ?it/s]

EPOCH : 1/80


100%|██████████| 190/190 [01:42<00:00,  1.85it/s]


train Loss: 0.4203 Acc: 0.8849
Training complete in 1m 42s


100%|██████████| 79/79 [00:28<00:00,  2.77it/s]


...

100%|██████████| 79/79 [00:27<00:00,  2.83it/s]


valid Loss: 0.2829 Acc: 0.9243
Training complete in 0m 28s


  0%|          | 0/190 [00:00<?, ?it/s]

EPOCH : 15/80


 23%|██▎       | 43/190 [00:24<01:19,  1.85it/s]

In [ ]:
#training on imagenette dataset
#Upto epochs 37 with lr = .0003
print(torch.cuda.is_available())
model = Attn_Graph()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = .0003)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
Learner_obj = Learner(datasets,model,criterion,optimizer,bs=50,num_workers=4,device=device)
epochs = 80
Learner_obj.fit(epochs=epochs)

  0%|          | 0/190 [00:00<?, ?it/s]

True
EPOCH : 1/80


  0%|          | 0/79 [00:00<?, ?it/s]

train Loss: 1.4373 Acc: 0.4985
Training complete in 1m 13s


  0%|          | 0/190 [00:00<?, ?it/s]

...

train Loss: 0.1744 Acc: 0.9409
Training complete in 1m 11s


  0%|          | 0/190 [00:00<?, ?it/s]

valid Loss: 0.5551 Acc: 0.8543
Training complete in 0m 25s
EPOCH : 37/80


 16%|█▋        | 31/190 [00:12<00:54,  2.94it/s]

KeyboardInterrupt: ignored

In [ ]:
#training on imagenette dataset
#from epochs 37 to 47 with lr = .000001
print(torch.cuda.is_available())
model = Attn_Graph()
model.load_state_dict(torch.load('checkpoint.pth'))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = .000001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
Learner_obj = Learner(datasets,model,criterion,optimizer,bs=50,num_workers=4,device=device)
epochs = 80
Learner_obj.fit(epochs=epochs)

  0%|          | 0/190 [00:00<?, ?it/s]

True
EPOCH : 1/80


  0%|          | 0/79 [00:00<?, ?it/s]

train Loss: 0.1586 Acc: 0.9460
Training complete in 1m 10s


  0%|          | 0/190 [00:00<?, ?it/s]

...

train Loss: 0.1177 Acc: 0.9608
Training complete in 1m 11s


  0%|          | 0/190 [00:00<?, ?it/s]

valid Loss: 0.4186 Acc: 0.8864
Training complete in 0m 25s
EPOCH : 11/80


 22%|██▏       | 42/190 [00:16<00:44,  3.36it/s]

KeyboardInterrupt: ignored

In [ ]:
print(torch.cuda.is_available())
model = Attn_Graph()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = .0003)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
Learner_obj = Learner(datasets,model,criterion,optimizer,bs=256,num_workers=4,device=device)
epochs = 80
Learner_obj.fit(epochs=epochs)

  0%|          | 0/313 [00:00<?, ?it/s]

True
EPOCH : 1/80


  0%|          | 0/79 [00:00<?, ?it/s]

train Loss: 4.2221 Acc: 0.1085
Training complete in 0m 38s


  0%|          | 0/313 [00:00<?, ?it/s]

...

train Loss: 2.5425 Acc: 0.3855
Training complete in 0m 37s


  0%|          | 0/313 [00:00<?, ?it/s]

valid Loss: 3.1046 Acc: 0.2996
Training complete in 0m 8s
EPOCH : 21/80


 27%|██▋       | 83/313 [00:10<00:24,  9.46it/s]

KeyboardInterrupt: ignored

In [ ]:
print(torch.cuda.is_available())
model = Attn_Graph()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = .0003)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
Learner_obj = Learner(datasets,model,criterion,optimizer,bs=256,num_workers=4,device=device)
epochs = 80
Learner_obj.fit(epochs=epochs)

  0%|          | 0/313 [00:00<?, ?it/s]

True
EPOCH : 1/80


  0%|          | 0/79 [00:00<?, ?it/s]

train Loss: 3.7911 Acc: 0.1791
Training complete in 0m 56s


  0%|          | 0/313 [00:00<?, ?it/s]

...

train Loss: 1.5590 Acc: 0.5947
Training complete in 0m 54s


  0%|          | 0/313 [00:00<?, ?it/s]

valid Loss: 2.4781 Acc: 0.4315
Training complete in 0m 12s
EPOCH : 19/80


 58%|█████▊    | 180/313 [00:31<00:21,  6.30it/s]

KeyboardInterrupt: ignored